# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770309760916                   -0.53    9.0         
  2   -2.771681627988       -2.86       -1.31    1.0    101ms
  3   -2.771714098549       -4.49       -2.60    1.0    121ms
  4   -2.771714711633       -6.21       -3.94    2.0    127ms
  5   -2.771714714870       -8.49       -4.25    2.0    105ms
  6   -2.771714715242       -9.43       -5.52    1.0    109ms
  7   -2.771714715250      -11.11       -6.17    2.0    105ms
  8   -2.771714715250      -13.46       -6.81    1.0    111ms
  9   -2.771714715250   +  -14.45       -7.37    2.0    120ms
 10   -2.771714715250      -14.15       -7.74    1.0    110ms
 11   -2.771714715250   +  -15.35       -8.75    1.0   97.1ms


-0.0001345727018044265

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770554569407                   -0.52    9.0         
  2   -2.771781060383       -2.91       -1.32    1.0   89.2ms
  3   -2.771801728359       -4.68       -2.46    1.0    122ms
  4   -2.771802006937       -6.56       -3.16    1.0    115ms
  5   -2.771802074119       -7.17       -4.16    2.0    103ms
  6   -2.771802074282       -9.79       -4.35    1.0    110ms
  7   -2.771802074474       -9.72       -5.55    1.0   94.9ms
  8   -2.771802074476      -11.67       -5.73    2.0    121ms
  9   -2.771802074476      -13.81       -6.17    1.0    102ms
 10   -2.771802074476      -13.51       -6.77    1.0    100ms
 11   -2.771802074476      -13.91       -7.29    1.0    107ms
 12   -2.771802074476   +  -13.58       -7.49    2.0    129ms
 13   -2.771802074476      -14.27       -7.69    1.0    109ms
 14   -2.771802074476      -13.89       -8.49    1.0    114ms


0.017612221477112146

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345727018044265
Displaced dipole:  0.017612221477112146
Polarizability :   1.7746794178916572


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920757871e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553298005e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852761759818e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694601484728e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088784529339e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.274905312325e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.133040499916e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.133040499916e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 5.149393251139e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.729826316605e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.217140135519e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.217140135519e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.2171837534319207e-12
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.